In [ ]:
import pandas as pd
import numpy as np
from sglang import function, system, user, assistant, gen, set_default_backend, RuntimeEndpoint
import sglang as sgl
from torch import save


set_default_backend(RuntimeEndpoint("http://localhost:30000"))

@sgl.function
def text_qa(s, question):
    s += question
    s += sgl.gen("answer", stop="]", max_tokens=100,temperature=0)

# Step 1: Read the inputs.csv file and sample 50 entries
df_inputs = pd.read_csv('inputs.csv') # index,date,input
#df_inputs = df_inputs.sample(50)



In [ ]:
df_inputs.sample(1)

In [ ]:
# Function to split DataFrame into chunks
def split_dataframe(df, chunk_size):
    # Use np.array_split to handle uneven sizes automatically
    chunks = np.array_split(df, len(df) // chunk_size + (1 if len(df) % chunk_size else 0))
    return chunks

# Split the DataFrame into chunks of size 5
# df_chunks = split_dataframe(df_inputs, 2000)
df_chunks = split_dataframe(df_inputs, 200)

for thing in df_chunks:
    print(len(thing))

In [ ]:
for i, df_chunk in enumerate(df_chunks):
    inputs = df_chunk['input'].tolist()
    print("<s>"+inputs[0]+ """
```json
{
  "category": \"""")
    break

In [ ]:
# Process each chunk
for i, df_chunk in enumerate(df_chunks):
    output_loc = f'outputs/outputs_chunk_{i+1}.csv'
    import os
    if os.path.exists(output_loc):
        print(f"Skipping chunk {i+1} as it already exists")
        continue
    inputs = df_chunk['input'].tolist()
    dates = df_chunk['date'].tolist()
    indices = df_chunk['index'].tolist()
    
    # Run batch inference for the current chunk
    states = text_qa.run_batch(
        [{"question": 
("<s>"+input+"""
```json
{
  "category": \"""")
         } for input in inputs],
        progress_bar=True
    )
    
    texts = []
    for state in states:
        for out in state.text_iter():
            texts.append(out.split("Sure! Here is the requested JSON document:")[1].strip())
    
    # Save the outputs of the current chunk to a separate file
    # save(texts, f'outputs_chunk_{i+1}.pt')
    chunk_df = pd.DataFrame({
        'index': indices,
        'date': dates,
        'output': texts
    })
    import os
    os.makedirs('outputs', exist_ok=True)
    chunk_df.to_csv(output_loc, index=False)

In [ ]:
h